## Loading Data

In [58]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [59]:
### Load enrolled and testing data

In [60]:
img_enrolled = cv2.imread('C:/Users/Mechanic/Iris Recognition/Test/Img_012_L_2.bmp')
img_test_1   = cv2.imread('C:/Users/Mechanic/Iris Recognition/Test/Img_012_L_3.bmp')
img_test_2   = cv2.imread('C:/Users/Mechanic/Iris Recognition/Test/Img_028_R_2.bmp')

## Pre-Processing and finding pupil

In [64]:
def pre_processing(image):
    img = image[:, :, 0].copy()
    img[img > 225] = 30
    img = cv2.medianBlur(img, 21)
    circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=10, maxRadius=200)
    circles = np.uint16(np.around(circles))
    x = circles[0, 0][0]
    y = circles[0, 0][1]
    r = circles[0, 0][2]
    return img, x, y, r

In [65]:
enr, enr_x, enr_y, enr_r = pre_processing(img_enrolled)
test1, test_1_x, test_1_y, test_1_r   = pre_processing(img_test_1)
test2, test_2_x, test_2_y, test_2_r = pre_processing(img_test_2)

## Finding iris 

In [66]:
def find_iris_id(img, x, y, r):
    x, y, r, l = find_segment(img, x, y, minr=max(int(1.25 * r), 100),
                              sigma=5, center_margin=30, jump=5)
    x, y, r, l = find_segment(img, x, y, minr=r - 10, maxr=r + 10,
                              sigma=2, center_margin=5, jump=1)
    return x, y, r

In [67]:
def find_segment(img, x0, y0, minr=0, maxr=500, step=1, sigma=5., center_margin=30, segment_type='iris', jump=1):
    max_o = 0
    max_l = []

    if img.ndim > 2:
        img = img[:, :, 0]
    margin_img = np.pad(img, maxr, 'edge')
    x0 += maxr
    y0 += maxr
    for x in range(x0 - center_margin, x0 + center_margin + 1, jump):
        for y in range(y0 - center_margin, y0 + center_margin + 1, jump):
            if segment_type == 'pupil':
                l = np.array([integrate(margin_img, y, x, r) for r in range(minr, maxr, step)])
            else:
                l = np.array([integrate(margin_img, y, x, r, 1 / 8, 3 / 8, n=8) +
                              integrate(margin_img, y, x, r, 5 / 8, 7 / 8, n=8)
                              for r in range(minr + abs(x0 - x) + abs(y0 - y), maxr, step)])
            l = (l[2:] - l[:-2]) / 2
            l = gaussian_filter(l, sigma)
            l = np.abs(l)
            max_c = np.max(l)
            if max_c > max_o:
                max_o = max_c
                max_l = l
                max_x, max_y = x, y
                r = np.argmax(l) * step + minr + abs(x0 - x) + abs(y0 - y)

    return max_x - maxr, max_y - maxr, r, max_l

In [68]:
import math
from scipy.ndimage.filters import gaussian_filter

In [69]:
def integrate(img, x0, y0, r, arc_start=0, arc_end=1, n=10):
    theta = 2 * math.pi / n
    integral = 0
    for step in np.arange(arc_start * n, arc_end * n, arc_end - arc_start):
        x = int(x0 + r * math.cos(step * theta))
        y = int(y0 + r * math.sin(step * theta))
        integral += img[x, y]
    return integral / n

In [70]:
enr_x_iris, enr_y_iris, enr_r_iris       = find_iris_id(enr, enr_x, enr_y, enr_r)
test1_x_iris, test1_y_iris, test1_r_iris = find_iris_id(test1, test_1_x, test_1_y, test_1_r)
test2_x_iris, test2_y_iris, test2_r_iris = find_iris_id(test2, test_2_x, test_2_y, test_2_r)

## Converting Polar to Cart

In [72]:
def polar2cart(r, x0, y0, theta):
    x = int(x0 + r * math.cos(theta))
    y = int(y0 + r * math.sin(theta))
    return x, y

In [73]:
def unravel_iris(image, xp, yp, rp, xi, yi, ri):
    if image.ndim > 2:
        image = image[:, :, 0].copy()
    iris = np.zeros((150, 300))
    theta = np.linspace(0, 2 * np.pi, 300)
    for i in range(300):
        begin = polar2cart(rp, xp, yp, theta[i])
        end = polar2cart(ri, xi, yi, theta[i])
        xspace = np.linspace(begin[0], end[0], 150)
        yspace = np.linspace(begin[1], end[1], 150)
        iris[:, i] = [255 - image[int(y), int(x)]
                      if 0 <= int(x) < image.shape[1] and 0 <= int(y) < image.shape[0]
                      else 0
                      for x,y in zip(xspace, yspace)]
    return iris

In [74]:
enr_iris   =  unravel_iris(img_enrolled, enr_x, enr_y, enr_r, enr_x_iris, enr_y_iris, enr_r_iris)
test1_iris =  unravel_iris(img_test_1, test_1_x, test_1_y, test_1_r, test1_x_iris, test1_y_iris, test1_r_iris)
test2_iris =  unravel_iris(img_test_2, test_2_x, test_2_y, test_2_r, test2_x_iris, test2_y_iris, test2_r_iris)

## Extracting Features

In [76]:
from skimage.util import view_as_blocks

In [77]:
def gabor_convolve(img, w, alpha, beta):
    rho = np.array([np.linspace(0, 1, img.shape[0]) for i in range(img.shape[1])]).T
    x = np.linspace(0, 1, img.shape[0])
    y = np.linspace(-np.pi, np.pi, img.shape[1])
    xx, yy = np.meshgrid(x, y)
    qwerty = np.exp(-w * 1j * (0 - yy)) * np.exp(-(xx - 0.5) ** 2 / alpha ** 2) * \
                                                    np.exp(-(yy - 0) ** 2 / beta ** 2)
    return rho * img * np.real(qwerty.T), \
           rho * img * np.imag(qwerty.T)

In [78]:
def iris_encode(img, dr=15, dtheta=15, alpha=0.4):
    """Encodes the straightened representation of an iris with gabor wavelets.

    :param img: Image of an iris
    :param dr: Width of image patches producing one feature
    :param dtheta: Length of image patches producing one feature
    :param alpha: Gabor wavelets modifier (beta parameter of Gabor wavelets becomes inverse of this number)
    :return: Iris code and its mask
    :rtype: tuple (ndarray, ndarray)
    """
    # mean = np.mean(img)
    # std = img.std()
    mask = view_as_blocks(np.logical_and(100 < img, img < 230), (dr, dtheta))
    norm_iris = (img - img.mean()) / img.std()
    patches = view_as_blocks(norm_iris, (dr, dtheta))
    code = np.zeros((patches.shape[0] * 3, patches.shape[1] * 2))
    code_mask = np.zeros((patches.shape[0] * 3, patches.shape[1] * 2))
    for i, row in enumerate(patches):
        for j, p in enumerate(row):
            for k, w in enumerate([8, 16, 32]):
                wavelet = gabor_convolve(p, w, alpha, 1 / alpha)
                code[3 * i + k, 2 * j] = np.sum(wavelet[0])
                code[3 * i + k, 2 * j + 1] = np.sum(wavelet[1])
                code_mask[3 * i + k, 2 * j] = code_mask[3 * i + k, 2 * j + 1] = \
                    1 if mask[i, j].sum() > dr * dtheta * 3 / 4 else 0
    code[code >= 0] = 1
    code[code < 0] = 0
    return code, code_mask

In [79]:
enr_code, enr_mask = iris_encode(enr_iris)
test1_code, test1_mask = iris_encode(test1_iris)
test2_code, test2_mask = iris_encode(test2_iris)

## Finding similairity using Jaccard Index

In [81]:
def compare_codes(a, b, mask_a, mask_b, rotation=False):
    """Compares two codes and calculates Jaccard index.

    :param a: Code of the first iris
    :param b: Code of the second iris
    :param mask_a: Mask of the first iris
    :param mask_b: Mask of the second iris
    :param rotation: Maximum cyclic rotation of the code. If this argument is greater than zero, the function will
        return minimal distance of all code rotations. If this argument is False, no rotations are calculated.

    :return: Distance between two codes.
    """
    if rotation:
        d = []
        for i in range(-rotation, rotation + 1):
            c = np.roll(b, i, axis=1)
            mask_c = np.roll(mask_b, i, axis=1)
            d.append(np.sum(np.remainder(a + c, 2) * mask_a * mask_c) / np.sum(mask_a * mask_c))
        return np.min(d)
    return np.sum(np.remainder(a + b, 2) * mask_a * mask_b) / np.sum(mask_a * mask_b)

In [82]:
similiarity = compare_codes(enr_code, test1_code, enr_mask, test1_mask)
similiarity2 = compare_codes(enr_code, test2_code, enr_mask, test2_mask)

In [83]:
print (similiarity)
print (similiarity2)

0.315
0.5025252525252525
